# Binned Rank Winrate Model

## Overview
This notebook will outline a simple method to generate predictions based on the players' most recent rank. We will bin the ranks and calculate the historic winrates between different ranks. For example, what is the historic winrate for a player ranked between 1-5th and 15th-25th? We can use this to generate predictions for every possible match up in the Australian Open. In this example we will look at both the mens and womens draw.

In [260]:
# Import libraries
import pandas as pd
import numpy as np

In [261]:
# Read in both datasets
mens = pd.read_csv('data/ATP_matches.csv', na_values='.').dropna()
womens = pd.read_csv('data/WTA_matches.csv', na_values='.').dropna()

In [263]:
mens.head(2)

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_DoubleFaults,Loser_FirstServes_Won,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,2-Jan-12,Hard,First Round,106.0,97.0,0,2.0,...,3.0,21.0,33.0,13.0,26.0,1.0,3.0,15.0,49.0,49
1,Dudi Sela,Fabio Fognini,Chennai,2-Jan-12,Hard,First Round,83.0,48.0,0,2.0,...,4.0,17.0,32.0,5.0,26.0,0.0,1.0,8.0,33.0,30


Logical bins based on ranking are 1st-5th, 6th-15th, 16th-39th, 40th-100th, 101st-200th and 200th-10000th. We will use these as our groups to calculate historic winrates for.

In [294]:
best_bin = [0, 5, 15, 40, 100, 200, 10000]

# Define a function which cuts the main dfs rank columns into their bins. e.g. if a player is ranked 4th, this function
# will output their bin as "5" as this is the cutoff
def get_binned_df(df, bins):
    # Cut the rank for both the winner and the loser
    df['winner_rank_bin'] = pd.cut(df.Winner_Rank.astype(int), 
           bins=bins,
          labels=bins[1:])

    df['loser_rank_bin'] = pd.cut(df.Loser_Rank.astype(int), 
           bins=bins,
          labels=bins[1:])
    return df

# Define a function which calculates the winrates between bins and returns a dictionary of these winrates
def get_winrates_between_groups(grouped_df, bins):
    # Get win rates between groups
    win_rates = {}
    
    # Loop over the bins
    for i in bins[1:]:
        for j in bins[1:]:
            # Get the total amount of wins for each group against another group
            # e.g. 1-5th vs 15th-25th may have 49 wins and 15th-25th vs 1-5th may have 30 wins
            winner_bin_wins = grouped_df[(grouped_df.winner_rank_bin == i) & (grouped_df.loser_rank_bin == j)].Winner.values[0]
            loser_bin_losses = grouped_df[(grouped_df.loser_rank_bin == i) & (grouped_df.winner_rank_bin == j)].Winner.values[0]
            
            # From this we can calculate the win percentage
            # e.g. 1-5th would have a win percentage of 49 / (49 + 30)
            
            win_pc = winner_bin_wins / (loser_bin_losses + winner_bin_wins)
            lose_pc = loser_bin_losses / (loser_bin_losses + winner_bin_wins)
            
            # Add this percentage to the win_rates dictionary
            win_rates[str(i) + ' vs ' + str(j)] = win_pc
            win_rates[str(j) + ' vs ' + str(i)] = lose_pc
    return win_rates

In [300]:
# Create the binned ranks in a new df
binned_df = get_binned_df(mens, best_bin)

# Get count of wins and losses for each group vs other groups
mens_groups = mens.groupby(['winner_rank_bin', 'loser_rank_bin'], as_index=False).Winner.count()

In [301]:
mens_groups.head(2)

,winner_rank_bin,loser_rank_bin,Winner
0,5,5,123
1,5,15,278


In [302]:
# Get win rates dict
win_rates = get_winrates_between_groups(mens_groups, best_bin)

In [303]:
win_rates

{'100 vs 100': 0.5,
 '100 vs 10000': 0.6904109589041096,
 '100 vs 15': 0.22521915037086987,
 '100 vs 200': 0.5884578997161779,
 '100 vs 40': 0.3685706166524581,
 '100 vs 5': 0.10951008645533142,
 '10000 vs 100': 0.3095890410958904,
 '10000 vs 10000': 0.5,
 '10000 vs 15': 0.16793893129770993,
 '10000 vs 200': 0.3732824427480916,
 '10000 vs 40': 0.1636904761904762,
 '10000 vs 5': 0.03225806451612903,
 '15 vs 100': 0.7747808496291302,
 '15 vs 10000': 0.8320610687022901,
 '15 vs 15': 0.5,
 '15 vs 200': 0.8282608695652174,
 '15 vs 40': 0.6788321167883211,
 '15 vs 5': 0.23835616438356164,
 '200 vs 100': 0.41154210028382215,
 '200 vs 10000': 0.6267175572519084,
 '200 vs 15': 0.17173913043478262,
 '200 vs 200': 0.5,
 '200 vs 40': 0.2988013698630137,
 '200 vs 5': 0.09693877551020408,
 '40 vs 100': 0.631429383347542,
 '40 vs 10000': 0.8363095238095238,
 '40 vs 15': 0.32116788321167883,
 '40 vs 200': 0.7011986301369864,
 '40 vs 40': 0.5,
 '40 vs 5': 0.1822429906542056,
 '5 vs 100': 0.890489913544

In [306]:
# Read dummy submissions file
submissions = pd.read_csv('data/men_dummy_submission_file.csv')

# Reshape the df to long by appending the loser data to the winner data
winner_df = mens[['Winner', 'Tournament_Date', 'Winner_Rank']].rename(columns={'Winner': 'Player', 'Winner_Rank': 'Rank'})
loser_df = mens[['Loser', 'Tournament_Date', 'Loser_Rank']].rename(columns={'Loser': 'Player', 'Loser_Rank': 'Rank'})
long_df = winner_df.append(loser_df)

# Get a df of the players most recent rank based on the last game they played
recent_rankings = long_df.groupby('Player', as_index=False).last()[['Player', 'Rank']]

# Merge this data to the submissions df
submissions_with_rank = (submissions.pipe(pd.merge, 
                                          recent_rankings, 
                                          left_on='player_1', 
                                          right_on='Player', 
                                          suffixes=('', '_1'))
                         .pipe(pd.merge, recent_rankings, left_on='player_2', right_on='Player',  suffixes=('', '_2'))
                         .drop(columns=['Player', 'Player_2']))


# Get the players' rank groups
submissions_with_rank['player_1_rank'] = pd.cut(submissions_with_rank.Rank.astype(int), 
                                                bins=best_bin,
                                                labels=best_bin[1:])

submissions_with_rank['player_2_rank'] = pd.cut(submissions_with_rank.Rank_2.astype(int), 
                                                bins=best_bin,
                                                labels=best_bin[1:])

submissions_with_rank['player_1_vs_player_2_rank'] = submissions_with_rank.player_1_rank.astype(str) + ' vs ' + submissions_with_rank.player_2_rank.astype(str)

# Map win rates from dict to df
submissions_with_rank['player_1_win_probability'] = submissions_with_rank.player_1_vs_player_2_rank.map(win_rates).fillna(0)

In [307]:
# Create submissions df
mens_submission_df = submissions_with_rank[['player_1', 'player_2', 'player_1_win_probability']].copy()

In [315]:
mens_submission_df.head(10)

,player_1,player_2,player_1_win_probability
0,Novak Djokovic,Rafael Nadal,0.500000
1,Roger Federer,Rafael Nadal,0.500000
2,Juan Martin del Potro,Rafael Nadal,0.500000
3,Alexander Zverev,Rafael Nadal,0.500000
4,Kevin Anderson,Rafael Nadal,0.238356
5,Marin Cilic,Rafael Nadal,0.238356
6,Dominic Thiem,Rafael Nadal,0.238356
7,Kei Nishikori,Rafael Nadal,0.238356
8,John Isner,Rafael Nadal,0.238356
9,Karen Khachanov,Rafael Nadal,0.182243


Now let's repeat the process for the women's data

In [319]:
# Create the binned ranks in a new df
womens_binned_df = get_binned_df(womens, best_bin)

# Get count of wins and losses for each group vs other groups
womens_groups = womens.groupby(['winner_rank_bin', 'loser_rank_bin'], as_index=False).Winner.count()

# Get win rates dict
womens_win_rates = get_winrates_between_groups(womens_groups, best_bin)

# Read dummy submissions file
womens_submissions = pd.read_csv('data/women_dummy_submission_file.csv', encoding='latin1')

# Reshape the df to long by appending the loser data to the winner data
winner_df = womens[['Winner', 'Tournament_Date', 'Winner_Rank']].rename(columns={'Winner': 'Player', 'Winner_Rank': 'Rank'})
loser_df = womens[['Loser', 'Tournament_Date', 'Loser_Rank']].rename(columns={'Loser': 'Player', 'Loser_Rank': 'Rank'})
long_df = winner_df.append(loser_df)

# Get a df of the players most recent rank based on the last game they played
recent_rankings = long_df.groupby('Player', as_index=False).last()[['Player', 'Rank']]

# Merge this data to the submissions df
submissions_with_rank = (womens_submissions.pipe(pd.merge, 
                                          recent_rankings, 
                                          left_on='player_1', 
                                          right_on='Player', 
                                          suffixes=('', '_1'))
                         .pipe(pd.merge, recent_rankings, left_on='player_2', right_on='Player',  suffixes=('', '_2'))
                         .drop(columns=['Player', 'Player_2']))


# Get the players' rank groups
submissions_with_rank['player_1_rank'] = pd.cut(submissions_with_rank.Rank.astype(int), 
                                                bins=best_bin,
                                                labels=best_bin[1:])

submissions_with_rank['player_2_rank'] = pd.cut(submissions_with_rank.Rank_2.astype(int), 
                                                bins=best_bin,
                                                labels=best_bin[1:])

submissions_with_rank['player_1_vs_player_2_rank'] = submissions_with_rank.player_1_rank.astype(str) + ' vs ' + submissions_with_rank.player_2_rank.astype(str)

# Map win rates from dict to df
submissions_with_rank['player_1_win_probability'] = submissions_with_rank.player_1_vs_player_2_rank.map(womens_win_rates).fillna(0)

In [320]:
submissions_with_rank.head(10)

,player_1,player_2,player_1_win_probability,Rank,Rank_2,player_1_rank,player_2_rank,player_1_vs_player_2_rank
0,Simona Halep,Angelique Kerber,0.500000,1.0,3.0,5,5,5 vs 5
1,Caroline Wozniacki,Angelique Kerber,0.500000,2.0,3.0,5,5,5 vs 5
2,Elina Svitolina,Angelique Kerber,0.500000,5.0,3.0,5,5,5 vs 5
3,Naomi Osaka,Angelique Kerber,0.414773,6.0,3.0,15,5,15 vs 5
4,Sloane Stephens,Angelique Kerber,0.414773,6.0,3.0,15,5,15 vs 5
5,Petra Kvitova,Angelique Kerber,0.500000,4.0,3.0,5,5,5 vs 5
6,Karolina Pliskova,Angelique Kerber,0.414773,8.0,3.0,15,5,15 vs 5
7,Kiki Bertens,Angelique Kerber,0.414773,9.0,3.0,15,5,15 vs 5
8,Aryna Sabalenka,Angelique Kerber,0.414773,11.0,3.0,15,5,15 vs 5
9,Anastasija Sevastova,Angelique Kerber,0.414773,11.0,3.0,15,5,15 vs 5
